In [1]:
from pyflink.table import EnvironmentSettings, TableEnvironment
import os
import json
from pyflink.table.expressions import *
from pyflink.table.window import *

# Line only for Local Development
os.environ["IS_LOCAL"] = "True"


In [6]:
# 1. Creates a Table Environment
env_settings = EnvironmentSettings.in_streaming_mode()
table_env = TableEnvironment.create(env_settings)

statement_set = table_env.create_statement_set()

APPLICATION_PROPERTIES_FILE_PATH = "/etc/flink/application_properties.json"  # on Kinesis Data Analytics

is_local = (
    True if os.environ.get("IS_LOCAL") else False
)  # set this env var in your local environment

if is_local:
    # only for local, overwrite variable to properties and pass in your jars delimited by a semicolon (;)
    print("Running Flink locally...")
    APPLICATION_PROPERTIES_FILE_PATH = "application_properties.json"  # local

    CURRENT_DIR = os.path.abspath(os.getcwd())
    table_env.get_config().set(
        "pipeline.jars",
        f"file:///{CURRENT_DIR}/lib/auxiliar-scripts/apache-flink/tutorial-aws-flink/kinesis-stream-to-stream/pomtest/target/aws-kinesis-analytics-python-apps-1.jar")
    print(f"file:///{CURRENT_DIR}/lib/auxiliar-scripts/apache-flink/tutorial-aws-flink/kinesis-stream-to-stream/pomtest/target/flink-sql-connector-kinesis-1.15.4.jar")


Running Flink locally...
file:////Users/viniciusdeoliveiramartucci/Documents/GitHub/martucci-glue-streaming/auxiliar-scripts/apache-flink/tutorial-aws-flink/kinesis-stream-to-stream/lib/auxiliar-scripts/apache-flink/tutorial-aws-flink/kinesis-stream-to-stream/pomtest/target/aws-kinesis-analytics-python-apps-1.jar


In [3]:
def get_application_properties():
    if os.path.isfile(APPLICATION_PROPERTIES_FILE_PATH):
        with open(APPLICATION_PROPERTIES_FILE_PATH, "r") as file:
            contents = file.read()
            properties = json.loads(contents)
            return properties
    else:
        print('A file at "{}" was not found'.format(APPLICATION_PROPERTIES_FILE_PATH))


def property_map(props, property_group_id):
    for prop in props:
        if prop["PropertyGroupId"] == property_group_id:
            return prop["PropertyMap"]

def create_source_table(table_name, stream_name, region, stream_initpos):
    return """ CREATE TABLE IF NOT EXISTS {0} (
                ticker VARCHAR(6),
                price DOUBLE,
                event_time TIMESTAMP(3),
                WATERMARK FOR event_time AS event_time - INTERVAL '5' SECOND

              )
              PARTITIONED BY (ticker)
              WITH (
                'connector' = 'kinesis',
                'stream' = '{1}',
                'aws.region' = '{2}',
                'scan.stream.initpos' = '{3}',
                'format' = 'json',
                'json.timestamp-format.standard' = 'ISO-8601'
              ) """.format(
        table_name, stream_name, region, stream_initpos
    )


def create_print_table(table_name, stream_name, region, stream_initpos):
    return """ CREATE TABLE IF NOT EXISTS {0} (
                ticker VARCHAR(6),
                price DOUBLE,
                event_time TIMESTAMP(3),
                WATERMARK FOR event_time AS event_time - INTERVAL '5' SECOND

              )
              WITH (
                'connector' = 'print'
              ) """.format(
        table_name, stream_name, region, stream_initpos
    )

def create_output_table(table_name, stream_name, region):
    return """ CREATE TABLE IF NOT EXISTS {0} (
                ticker VARCHAR(6),
                start_timestamp TIMESTAMP(3),
                end_timestamp TIMESTAMP(3),
                rowtime_timestamp TIMESTAMP(3),
                average_price DOUBLE

              )
              PARTITIONED BY (ticker)
              WITH (
                'connector' = 'kinesis',
                'stream' = '{1}',
                'aws.region' = '{2}',
                'sink.partitioner-field-delimiter' = ';',
                'sink.batch.max-size' = '100',
                'format' = 'json',
                'json.timestamp-format.standard' = 'ISO-8601'
              ) """.format(
        table_name, stream_name, region
    )

def create_fake_source_table(table_name):
    return """ CREATE TABLE IF NOT EXISTS {0} (
                ticker VARCHAR(6),
                price DOUBLE,
                event_time TIMESTAMP(3),
                WATERMARK FOR event_time AS event_time - INTERVAL '5' SECOND

              )
              PARTITIONED BY (ticker)
              WITH (
                'connector' = 'datagen',
                'number-of-rows' = '10'
              ) """.format(table_name)

def create_sink_table_s3(table_name, bucket_name):
    return """ CREATE TABLE {0} (
                ticker VARCHAR(6),
                price DOUBLE,
                event_time TIMESTAMP(3)
              )
              PARTITIONED BY (ticker)
              WITH (
                  'connector'='filesystem',
                  'path'='s3a://{1}/',
                  'format'='json',
                  'sink.partition-commit.policy.kind'='success-file',
                  'sink.partition-commit.delay' = '1 min'
              ) """.format(table_name, bucket_name)

In [4]:
#def main():
# Application Property Keys
input_property_group_key = "consumer.config.0"
producer_property_group_key = "producer.config.0"

input_stream_key = "input.stream.name"
input_region_key = "aws.region"
input_starting_position_key = "flink.stream.initpos"

output_stream_key = "output.stream.name"
output_region_key = "aws.region"

# tables
input_table_name = "input_table"
output_table_name = "output_table"

# get application properties
props = get_application_properties()

input_property_map = property_map(props, input_property_group_key)
output_property_map = property_map(props, producer_property_group_key)

input_stream = input_property_map[input_stream_key]
input_region = input_property_map[input_region_key]
stream_initpos = input_property_map[input_starting_position_key]

output_stream = output_property_map[output_stream_key]
output_region = output_property_map[output_region_key]

# 0. Creates fake source table from datagen
input_table_fake_name = "fake_source_table"
table_env.execute_sql(
    create_fake_source_table(input_table_fake_name)
)

# 2. Creates a source table from a Kinesis Data Stream
print(f" Creating source table {input_table_name} from source stream {input_stream} in region {input_region} using {stream_initpos}")
table_env.execute_sql(
    create_source_table(input_table_name, input_stream, input_region, stream_initpos)
)


# 3. Creates a print to check Data
print_output_table = output_table_name + "_print"
print(f" Creating print table {output_table_name} from source stream {input_stream} in region {input_region}.")
table_env.execute_sql(
    create_print_table(print_output_table, output_stream, output_region, stream_initpos)
)

# 4. Creates a sink table writing to a Kinesis Firehose Delivery Strem
print(f" Creating destination table {output_table_name} to destination stream {output_stream} in region {output_region}.")
table_env.execute_sql(
    create_output_table(output_table_name,output_stream, output_region)
)


# 5. Insert from Source to Destination/Print
table_result = table_env.execute_sql("INSERT INTO {0} SELECT * FROM {1}"
                                         .format(output_table_name, input_table_name))

table_result.wait()


 Creating source table input_table from source stream martucci-kinesis-stream-firehose in region us-east-1 using LATEST
 Creating print table output_table from source stream martucci-kinesis-stream-firehose in region us-east-1.
 Creating destination table output_table to destination stream martucci-kinesis-destination in region us-east-1.


Py4JJavaError: An error occurred while calling o8.executeSql.
: org.apache.flink.table.api.ValidationException: Unable to create a source for reading table 'default_catalog.default_database.input_table'.

Table options are:

'aws.region'='us-east-1'
'connector'='kinesis'
'format'='json'
'json.timestamp-format.standard'='ISO-8601'
'scan.stream.initpos'='LATEST'
'stream'='martucci-kinesis-stream-firehose'
	at org.apache.flink.table.factories.FactoryUtil.createDynamicTableSource(FactoryUtil.java:219)
	at org.apache.flink.table.factories.FactoryUtil.createDynamicTableSource(FactoryUtil.java:244)
	at org.apache.flink.table.planner.plan.schema.CatalogSourceTable.createDynamicTableSource(CatalogSourceTable.java:175)
	at org.apache.flink.table.planner.plan.schema.CatalogSourceTable.toRel(CatalogSourceTable.java:115)
	at org.apache.calcite.sql2rel.SqlToRelConverter.toRel(SqlToRelConverter.java:3997)
	at org.apache.calcite.sql2rel.SqlToRelConverter.convertIdentifier(SqlToRelConverter.java:2867)
